### Linear-Mixed Effects Model to Evaluate the Impact of Sampling Methods on Human Reading Times

Background:   
Recent findings show that larger and more accurate language models like
GPT-2, despite having lower perplexity, generates surprisal estimates that are not as effective in predicting human reading times and eye-gaze durations, challenging the “larger is better” trend in NLP.  

The objective of our project is to investigate the underlying causes of the diminished accuracy in predicting human reading times by more complex language models.  We hypothesize that appropriate **sampling methods** could potentially enhance the large language models’ performance in surprisal study, because sampling methods, such as top-k sampling, is implemented by zeroing out the probabilities for tokens below the k-th one, which will re-weight token logits (used
to calculate surprisal estimates) by removing noise. 

Our study is particularly focused on assessing whether the sampling methodologies influence the efficacy of the advanced language models in accurately predicting human reading times.

Authors: Meiyu (Emily) Li, Yuwen Shen, Tongyu Zhao, Zehui (Bella) Gu

#### Import Libraries

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

#### LME models and Evaluation

In [3]:
def combine_surp(df, df_surp):
    df_copy = df.copy()  
    df_copy['llm_surp'] = 0
    start_row = 0
    llmsurp_list = df_surp['llm_surp'].tolist()
    size_list = df_copy.groupby(['sentid', 'sentpos']).size().tolist()

    assert len(size_list) == len(df_surp)
    
    for i in range(len(df_surp)):
        df_copy.loc[start_row:start_row + size_list[i]-1, 'llm_surp'] = llmsurp_list[i]
        start_row = start_row + size_list[i]

    return df_copy

In [4]:
def preporcess(data):
    data['id'] = data.index
    data = data[data['sentpos'] != 1]
    ids = np.array([])
    last_sentpos_for_each_sent = data.groupby('sentid')['sentpos'].max().tolist()
    sentid_list = data['sentid'].unique().tolist()

    for i in range(len(last_sentpos_for_each_sent)):
        temp_data = data[data['sentid'] == sentid_list[i]]
        temp_id = temp_data[temp_data['sentpos'] == last_sentpos_for_each_sent[i]]['id'].tolist()
        ids = np.append(ids, temp_id)

    data = data[~data['id'].isin(ids)]
    data = data[data['correct'] >= 4]
    data = data[(data['WorkTimeInSeconds'] > 100) & (data['WorkTimeInSeconds'] < 3000)]

    # log_transform
    cols1 = ['wlen', 'sentpos', 'llm_surp', 'fdur']
    for col in cols1:
        if col in data.columns:
            data[col] = np.log(data[col]+1)
    # scale
    scaler = StandardScaler()
    predictors = ['wlen', 'sentpos', 'llm_surp', 'fdur']
    predictors = [col for col in predictors if col in data.columns]
    data[predictors] = scaler.fit_transform(data[predictors])
    return data

In [5]:
def main(model1, method1, model2, method2):
    raw_data = pd.read_csv('data/naturalstories.evmeasures', header = 0, sep = " ")
    surp1 = pd.read_csv(f'naturalstories.{model1}.{method1}.surprisal', header = 0, sep = " ", encoding = 'utf-16')
    surp2 = pd.read_csv(f'naturalstories.{model2}.{method2}.surprisal', header = 0, sep = " ", encoding = 'utf-16')
    
    data1 = combine_surp(raw_data, surp1)
    data2 = combine_surp(raw_data, surp2)
    data1 = preporcess(data1)
    data2 = preporcess(data2)
    print('Finish processing data!')
    
    baseline = smf.mixedlm("fdur ~ wlen + sentpos", data=data1, groups=data1["subject"])
    baseline = baseline.fit()
    ll_baseline = baseline.llf
    # print('Baseline: ')
    # print(baseline.summary())

    lme1 = smf.mixedlm("fdur ~ wlen + sentpos + llm_surp", data=data1, groups=data1["subject"])
    lme1 = lme1.fit()
    ll_model1 = lme1.llf
    # print(f'\n{model1}: ')
    # print(lme1.summary())

    lme2 = smf.mixedlm("fdur ~ wlen + sentpos + llm_surp", data=data2, groups=data2["subject"])
    lme2 = lme2.fit()
    ll_model2 = lme2.llf
    # print(f'\n{model2}: ')
    # print(lme2.summary())

    # print out the improvement of each model (with llm_surp) with respect to the baseline
    print(f'{model1}_{method1}: ', ll_model1 - ll_baseline)
    print(f'{model2}_{method2}: ', ll_model2 - ll_baseline, '\n')
    # return ll_baseline, ll_model1, ll_model2

In [6]:
main('neo125m', 'k100', 'neo125m', 'k1000')

Finish processing data!
neo125m_k100:  613.9199054753408
neo125m_k1000:  652.7866566646844 

